## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [40]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)


2.0.0


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
#tf.executing_eagerly()

### Collect Data

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('/gdrive/My Drive/Great_Lakes_Lab/neural_network/prices.csv')

### Check all columns in the dataset

In [48]:
data.shape

(851264, 7)

In [49]:
data.isna().sum()

date      0
symbol    0
open      0
close     0
low       0
high      0
volume    0
dtype: int64

In [50]:
data.isnull().sum()

date      0
symbol    0
open      0
close     0
low       0
high      0
volume    0
dtype: int64

In [51]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'], axis = 1, inplace= True)

In [53]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data1=data.iloc[0:1000,]

In [55]:
data1.shape

(1000, 5)

In [56]:
data1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [57]:
data1['volume']= data1['volume']/1000000

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
data1.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
data2=data1.values

In [60]:
type(data2)

numpy.ndarray

In [61]:
data2




array([[123.43    , 125.839996, 122.309998, 126.25    ,   2.1636  ],
       [125.239998, 119.980003, 119.940002, 125.540001,   2.3864  ],
       [116.379997, 114.949997, 114.93    , 119.739998,   2.4895  ],
       ...,
       [ 28.32    ,  28.77    ,  28.01    ,  28.809999,  37.1528  ],
       [ 44.      ,  44.799999,  43.75    ,  44.810001,   6.5686  ],
       [ 36.080002,  37.139999,  36.009998,  37.23    ,   5.6043  ]])

In [0]:
from sklearn.model_selection import train_test_split
X = data2[:,0:4]
Y = data2[:,4]
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [63]:
train_x.dtype


dtype('float64')

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

In [66]:
train_y.shape

(700,)

In [0]:
train_y=train_y.reshape(-1,1)

In [27]:
train_y.shape

(700, 1)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.Variable( tf.random.normal(shape=[4,3]))
b1 = tf.zeros(shape=(3))

In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.Variable( tf.random.normal(shape=[3,1]))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    #out=tf.sigmoid(net2)
    return net2
  


3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2 = prediction(train_x, w1, b1,w2,b2)
        current_loss = loss(net2,train_y)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    
    return w1, b1,w2,b2

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [75]:
for i in range(100):
    
    w1, b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)
    print("Loss at step {:d}: {:.3f}".format(i, loss(prediction(train_x, w1, b1,w2,b2), train_y)))

Loss at step 0: 241.999
Loss at step 1: 239.854
Loss at step 2: 237.877
Loss at step 3: 236.062
Loss at step 4: 234.400
Loss at step 5: 232.883
Loss at step 6: 231.502
Loss at step 7: 230.248
Loss at step 8: 229.112
Loss at step 9: 228.084
Loss at step 10: 227.155
Loss at step 11: 226.316
Loss at step 12: 225.559
Loss at step 13: 224.876
Loss at step 14: 224.260
Loss at step 15: 223.706
Loss at step 16: 223.205
Loss at step 17: 222.755
Loss at step 18: 222.349
Loss at step 19: 221.983
Loss at step 20: 221.653
Loss at step 21: 221.355
Loss at step 22: 221.088
Loss at step 23: 220.846
Loss at step 24: 220.629
Loss at step 25: 220.432
Loss at step 26: 220.256
Loss at step 27: 220.096
Loss at step 28: 219.953
Loss at step 29: 219.823
Loss at step 30: 219.707
Loss at step 31: 219.602
Loss at step 32: 219.507
Loss at step 33: 219.421
Loss at step 34: 219.344
Loss at step 35: 219.275
Loss at step 36: 219.213
Loss at step 37: 219.156
Loss at step 38: 219.105
Loss at step 39: 219.060
Loss at st

### Get the shapes and values of W and b

In [76]:
w1.shape

TensorShape([4, 3])

In [77]:
w1


<tf.Tensor: id=6728, shape=(4, 3), dtype=float32, numpy=
array([[-1.558792 ,  0.2873578,  0.5153978],
       [-0.2883653,  1.1513548,  1.2117592],
       [-1.9095633,  1.2070128,  0.6111313],
       [-0.3578395, -0.8021828, -0.9524116]], dtype=float32)>

In [78]:
b1.shape

TensorShape([3])

In [79]:
b1


<tf.Tensor: id=6731, shape=(3,), dtype=float32, numpy=array([ 0.09395523,  1.0405322 , -0.0234419 ], dtype=float32)>

In [80]:
w2.shape

TensorShape([3, 1])

In [81]:
w2

<tf.Tensor: id=6722, shape=(3, 1), dtype=float32, numpy=
array([[0.577008 ],
       [3.5736668],
       [0.6096489]], dtype=float32)>

In [82]:
b2.shape

TensorShape([1])

In [83]:
b2

<tf.Tensor: id=6725, shape=(1,), dtype=float32, numpy=array([2.0196707], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [84]:
test_data=test_x[0:1,:]
test_data

array([[0.49863145, 0.49986658, 0.49250004, 0.50886524]], dtype=float32)

In [85]:
pred=prediction(test_data, w1, b1,w2,b2)
pred

<tf.Tensor: id=6748, shape=(1, 1), dtype=float32, numpy=array([[5.619688]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data_iris = pd.read_csv('/gdrive/My Drive/Great_Lakes_Lab/neural_network/11_Iris.csv')

In [93]:
data_iris.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
data_iris.drop('Id',axis = 1,inplace=True)

In [95]:
data_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
y=pd.get_dummies(data_iris['Species'])


### Splitting the data into feature set and target set

In [0]:
X = data_iris.drop('Species', axis = 1)
Y = data_iris['Species']

In [97]:
X.shape

(150, 4)

In [98]:
Y.shape

(150,)

In [99]:
Y=np.array(Y).reshape(-1,1)
Y = pd.DataFrame(Y)
Y.shape

(150, 1)

In [0]:
Y['species'] = Y

In [0]:
Y = Y.drop(0, axis = 1)

In [105]:
Y.head()

,species
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa


In [106]:
Y =pd.get_dummies(Y)
Y.head()

,species_Iris-setosa,species_Iris-versicolor,species_Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [0]:
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

### Model Training 

In [159]:
print(tf.__version__)

2.0.0


In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(4,input_dim = 4,activation='sigmoid'))
model.add(tf.keras.layers.Dense(3,activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


### Model Prediction

In [123]:
output = model.fit(train_x, train_y,validation_data = (test_x,test_y), epochs = 50)
print(output)

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 4ms/sample - loss: 0.9262 - accuracy: 0.6667 - val_loss: 0.9279 - val_accuracy: 0.6667
Epoch 2/50
105/105 [==============================] - 0s 234us/sample - loss: 0.9236 - accuracy: 0.6667 - val_loss: 0.9251 - val_accuracy: 0.6667
Epoch 3/50
105/105 [==============================] - 0s 210us/sample - loss: 0.9216 - accuracy: 0.6667 - val_loss: 0.9219 - val_accuracy: 0.6667
Epoch 4/50
105/105 [==============================] - 0s 257us/sample - loss: 0.9194 - accuracy: 0.6667 - val_loss: 0.9198 - val_accuracy: 0.6667
Epoch 5/50
105/105 [==============================] - 0s 247us/sample - loss: 0.9176 - accuracy: 0.6667 - val_loss: 0.9174 - val_accuracy: 0.6667
Epoch 6/50
105/105 [==============================] - 0s 227us/sample - loss: 0.9159 - accuracy: 0.6667 - val_loss: 0.9151 - val_accuracy: 0.6667
Epoch 7/50
105/105 [==============================] - 0s 244us/sample - loss: 0.9

### Save the Model

In [115]:
model.save('iris_model.py')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: iris_model.py/assets


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?